------BFS----


In [1]:
from collections import deque

# Movement directions: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Check if cell is valid
def valid_cell(matrix, row, col, seen):
    rows, cols = len(matrix), len(matrix[0])
    return 0 <= row < rows and 0 <= col < cols and matrix[row][col] == 0 and (row, col) not in seen

# Build path from parent mapping
def build_route(parent_map, end_node):
    route = []
    while end_node in parent_map:
        route.append(end_node)
        end_node = parent_map[end_node]
    route.append(end_node)
    return route[::-1]

# BFS search function
def shortest_path_bfs(matrix, src, dest):
    q = deque([src])
    seen = set([src])
    parent_map = {}

    while q:
        curr_node = q.popleft()

        if curr_node == dest:
            path_found = build_route(parent_map, dest)
            return path_found, len(path_found) - 1

        for dr, dc in MOVES:
            nr, nc = curr_node[0] + dr, curr_node[1] + dc
            next_node = (nr, nc)

            if valid_cell(matrix, nr, nc, seen):
                q.append(next_node)
                seen.add(next_node)
                parent_map[next_node] = curr_node

    return None, -1  # No route

# Example run
if __name__ == "__main__":
    maze = [
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start_cell = (0, 0)
    end_cell = (4, 4)

    route, steps = shortest_path_bfs(maze, start_cell, end_cell)

    if route:
        print("Shortest Path:", route)
        print("Path Cost:", steps)
    else:
        print("No path found between start and goal.")


Shortest Path: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Path Cost: 8


-------DFS------

In [2]:
# Directions: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Check if a position is valid to visit
def valid_position(maze, r, c, visited_cells):
    rows, cols = len(maze), len(maze[0])
    return 0 <= r < rows and 0 <= c < cols and maze[r][c] == 0 and (r, c) not in visited_cells

# Build the path from parent references
def trace_path(parent_map, end_node):
    route = []
    while end_node in parent_map:
        route.append(end_node)
        end_node = parent_map[end_node]
    route.append(end_node)
    return route[::-1]

# DFS implementation
def depth_first_search(maze, start_point, target_point):
    stack_nodes = [start_point]
    visited_cells = set([start_point])
    parent_map = {}

    while stack_nodes:
        current_cell = stack_nodes.pop()

        if current_cell == target_point:
            route_found = trace_path(parent_map, target_point)
            return route_found, len(route_found) - 1

        for dr, dc in MOVES:
            nr, nc = current_cell[0] + dr, current_cell[1] + dc
            next_cell = (nr, nc)

            if valid_position(maze, nr, nc, visited_cells):
                stack_nodes.append(next_cell)
                visited_cells.add(next_cell)
                parent_map[next_cell] = current_cell

    return None, -1  # No route found

# Example run
if __name__ == "__main__":
    maze_layout = [
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start_cell = (0, 0)
    goal_cell = (4, 4)

    route, steps = depth_first_search(maze_layout, start_cell, goal_cell)

    if route:
        print("Path Found by DFS:", route)
        print("Path Cost:", steps)
    else:
        print("No path found from start to goal.")


Path Found by DFS: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Path Cost: 8


--------A*-------

In [3]:
import heapq

# Movement directions: up, down, left, right
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Manhattan distance heuristic
def manhattan_distance(pos_a, pos_b):
    return abs(pos_a[0] - pos_b[0]) + abs(pos_a[1] - pos_b[1])

# Check if a cell can be visited
def valid_move(maze, row, col):
    rows, cols = len(maze), len(maze[0])
    return 0 <= row < rows and 0 <= col < cols and maze[row][col] == 0

# Build path from parent mapping
def build_path(parent_map, end_cell):
    route = []
    while end_cell in parent_map:
        route.append(end_cell)
        end_cell = parent_map[end_cell]
    route.append(end_cell)
    return route[::-1]

# A* search implementation
def a_star_search(maze, start_cell, goal_cell):
    open_nodes = [(manhattan_distance(start_cell, goal_cell), 0, start_cell)]  # (f_score, g_score, position)
    heapq.heapify(open_nodes)

    parent_map = {}
    g_values = {start_cell: 0}
    visited_nodes = set()

    while open_nodes:
        _, g_cost, current_cell = heapq.heappop(open_nodes)

        if current_cell == goal_cell:
            route_found = build_path(parent_map, goal_cell)
            return route_found, g_cost

        if current_cell in visited_nodes:
            continue
        visited_nodes.add(current_cell)

        for dr, dc in MOVES:
            nr, nc = current_cell[0] + dr, current_cell[1] + dc
            neighbor_cell = (nr, nc)

            if valid_move(maze, nr, nc):
                tentative_g = g_cost + 1
                if neighbor_cell not in g_values or tentative_g < g_values[neighbor_cell]:
                    g_values[neighbor_cell] = tentative_g
                    f_score = tentative_g + manhattan_distance(neighbor_cell, goal_cell)
                    parent_map[neighbor_cell] = current_cell
                    heapq.heappush(open_nodes, (f_score, tentative_g, neighbor_cell))

    return None, -1  # No path found

# Example run
if __name__ == "__main__":
    maze_layout = [
        [0, 1, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start_pos = (0, 0)
    goal_pos = (4, 4)

    route, steps = a_star_search(maze_layout, start_pos, goal_pos)

    if route:
        print("Shortest Path Found by A*:", route)
        print("Path Cost:", steps)
    else:
        print("No path found from start to goal.")


Shortest Path Found by A*: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]
Path Cost: 8
